In [1]:
from matplotlib import pyplot as plt
import numpy as np
import re

plt.style.use('ggplot')

In [2]:
fileName = "new.csv"
numHeaderLines = 4
delimiter = ","
yDataNames = ["time", "RKneeOut_x", "RKneeOut_y","RKneeOut_z",
              "RKneeIn_x","RKneeIn_y","RKneeIn_z",
              "RThigh_x","RThigh_y", "RThigh_z", 
              "RAsis_x", "RAsis_y", "RAsis_z",
              "RPsi_x", "RPsi_y", "RPsi_z",
              "LAsis_x", "LAsis_y", "LAsis_z",
              "LPsi_x", "LPsi_y", "LPsi_z"]

regrexHeader = r'\w+'
Data = np.genfromtxt(fileName, delimiter=delimiter, skip_header=numHeaderLines+1)

with open(fileName) as infile:
    content = infile.readlines()
    row = content[numHeaderLines]
headers = row.split(delimiter)

dataIdx = {}
for i, header in enumerate(headers):
    x = re.findall(regrexHeader, header)
    if x: 
        if (x[0] in yDataNames):
            dataIdx.update({x[0] : i})

fileData = np.zeros((Data.shape[0], len(yDataNames)))
for i,name in enumerate(yDataNames):
    fileData[:,i] = Data[:, dataIdx[name]]

time = fileData[:, dataIdx["time"]]

In [3]:
def TransformationMatrix(points):
    P2P1 = points[1,:]-points[0,:]
    P3P1 = points[2,:]-points[0,:]
    
    xAxis = P2P1/np.linalg.norm(P2P1)
    yAxis_unNormalized = np.cross(P2P1, P3P1)
    yAxis = yAxis_unNormalized/np.linalg.norm(yAxis_unNormalized)
    zAxis = np.cross(xAxis, yAxis)
    
    origin = np.mean(points, axis=0)
    
    transformMat = np.column_stack((xAxis, yAxis, zAxis, origin))
    return np.row_stack((transformMat, np.array([0, 0, 0, 1])))

In [4]:
# For details refer to "Evaluation of a new algorithm to determine the hip joint center"
nRows = fileData.shape[0]
A = np.zeros((3*nRows, 6))
b = np.zeros((3*nRows, 1))

for i in range(nRows):
    femurGlobalVecs = np.reshape(fileData[i,1:10], (3,3))
    femur_transformMat = TransformationMatrix(femurGlobalVecs)
    
    pelvisGlobalVecs = np.reshape(fileData[i,10:-3], (3,3))
    pelvis_transformMat = TransformationMatrix(pelvisGlobalVecs)
    R1 = pelvis_transformMat[0:3, 0:3]
    R2 = femur_transformMat[0:3, 0:3]
    A[3*i:3*(i+1), :] = np.concatenate((np.identity(3), -np.matmul(R1.T, R2)), axis=1);
    
    femur_pelvis_pos_global = femur_transformMat[0:3,3] - pelvis_transformMat[0:3,3]
    femur_pelvis_pos = np.matmul(R1.T,femur_pelvis_pos_global)
    b[3*i:3*(i+1), 0] = femur_pelvis_pos

sHJC_lHJC = np.linalg.lstsq(A,b)[0]
print(sHJC_lHJC)

sHJC = np.concatenate((sHJC_lHJC[0:3,0], np.array([1])))
lHJC = np.concatenate((sHJC_lHJC[3:,0], np.array([1])))

pelvisGlobalVecs_t0 = np.reshape(fileData[1,10:-3], (3,3))
pelvis_transformMat_t0 = TransformationMatrix(pelvisGlobalVecs_t0)
sHJC_global = np.matmul(pelvis_transformMat_t0, sHJC)

femurGlobalVecs_t0 = np.reshape(fileData[1,1:10], (3,3))
femur_transformMat_t0 = TransformationMatrix(femurGlobalVecs_t0)
lHJC_global = np.matmul(femur_transformMat_t0, lHJC)

print(sHJC_global)
print(lHJC_global)

[[-0.05101634]
 [ 0.10549964]
 [ 0.11767206]
 [-0.04244561]
 [-0.00214877]
 [-0.31002945]]
[-0.05627235  0.4770067   0.14672127  1.        ]
[-0.05627235  0.4770067   0.14672127  1.        ]


<ipython-input-4-d72037899923>:20: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sHJC_lHJC = np.linalg.lstsq(A,b)[0]
